In [2]:
import numpy as np
from scipy.integrate import quad
from scipy.constants import epsilon_0

Question 1

In [3]:
def dEz_dtheta(theta,z):
    eps = 1
    sigma = 1
    R = 10
    coeff = (1/(4*np.pi*eps))*(2*np.pi*R**2*sigma)
    integrand = ((z-R*np.cos(theta))*np.sin(theta))/(R**2 + z**2 - 2*R*z*np.cos(theta))**(3/2)

    return coeff*integrand

In [9]:
z = np.linspace(0,10,1000)
Ez = np.zeros(len(z))

for i in  range(len(z)):
    def func(theta):
        return dEz_dtheta(theta, z[i])
    Ez[i] += quad(func,0,10)



ValueError: setting an array element with a sequence.

1.066780445414e-311